# Authentication and Initialization
To use NetBrain APIs, you need to authenticate user information to get access token. With the access token, you can specify a tenant/domain to get started with API calls.
<br><br>

In the Authentication category, the following APIs are included:
<br><br>

>* [Log in to get an authentication token](#Log-in-API)
<br><br>

>* [Log out from a session](#Log-out-API)
<br><br>

>* [Get all accessible tenants](#Get-all-accessible-tenants-API)
<br><br>

>* [Get all accessible domains of a tenant](#Get-all-accessible-domains-of-a-tenants-API)
<br><br>

>* [Specify a domain to work on](#Specify-a-domain-to-work-on-API)

# Log in API
Use this API call to get an authentication token and start a session. Authentication not required.
<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session  | No |
<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|username* | string  | the username to log into your NetBrain domain.  |
|password* | string  | the password to log into your NetBrain domain.  |
|authentication_id | string  | the authentication ID of an external user through SSO, LDAP/AD or TACACS. This parameter is only required for an external user and the value is alias. |
<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|token | string | The returned authentication token.  |
|statusCode| integer | The returned status code of executing the API.  |
|statusDescription| string | The explanation of the status code. |
<br><br>

* **Example**

In [12]:
# import python modules 
import requests
import time
import urllib3
import pprint
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set the request parameters
# Eg. User name="NetBrain", Password="Netbrain" for this code sample.
user = "gongdailiu"      
pwd = "1101810209"      
nb_url = "http://192.168.28.79"           

# define a login function 
def loginSession(nb_url, username, password):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}    
    try:
        # Do the HTTP request
        response = requests.post(full_url, headers=headers, auth=(username, password), verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Get token failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
js = loginSession(nb_url, user, pwd)
js# print out the token you got as a string 

{'token': 'fc6bc6ea-a46a-4e9b-8906-c623f78474b6',
 'statusCode': 790200,
 'statusDescription': 'Success.'}

In [ ]:
# For all APIs, when we get an error after call an API, the response only contains statuscode and description. That is unclear.
# is there any possible we can add another attribute to the response called "Message", like the following text:

# Whent he response shows like:
{
    'statusCode': 795000, 
    'statusDescription': 'Invalid username or password.'
}

# can we improve it as:
{
    "error" : {
        "code" : 795000,
        "Message" : "User login failed! Cannot get an access token.",
        "Status" : "Invalid username or password."
    }
}

#Because the status codes of our API system is different form the HTTP statuse codes, we should provide a table of status codes 

# Log out API
Use this API to log out from the current session.
<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **DELETE** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session  | Yes |
<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | input the token received from login.  |
<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | The returned status code of executing the API.  |
|statusDescription| string | The explanation of the status code. |
<br><br>

* **Example**


In [45]:
import json
# Set the request parameters
#token = "db389ceb-7988-4353-a398-9e04cff7e0b5"
nb_url = "http://192.168.28.79"

# Logout NetBrain and release system token.
def logoutSession(nb_url ,token):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    data = {"token":token}
    try:
        # Do the HTTP request
        response = requests.delete(full_url, data=json.dumps(data), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Session logout failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
result = logoutSession(nb_url ,token)
result# print out the result of whether log out successfully.

{'statusCode': 790200, 'statusDescription': 'Success.'}

In [ ]:
#Token should be consern as authentication header, not a body parameter.
# based on bearer auth concept:
# (Bearer authentication (also called token authentication) is an HTTP authentication scheme 
# that involves security tokens called bearer tokens. The name “Bearer authentication” can be understood as 
# “give access to the bearer of this token.” The bearer token is a cryptic string, usually generated by the server in response 
# to a login request. The client must send this token in the Authorization header when making requests to protected resources)
# we use Bearer Auth as our authentication. thus, we should consider token as a authentication header not a body parameter.

# Get all accessible tenants API
Use this function returns a list of accessible tenants (including tenant ID and names). The returned tenant list varies by the user privileges you use to log in. To retrieve a full list of all available tenants, you must log in with admin permissions. Authentication required.
<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Tenants | Yes |
<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|tenants | array | A list of all accessible tenants.  |
|tenantId| string | The tenant ID.  |
|tenantName| string | The tenant name. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |
<br><br>

* **Example**


In [13]:
# Set the request parameters
token = "fc6bc6ea-a46a-4e9b-8906-c623f78474b6"
nb_url = "http://192.168.28.79"

# Get all Tenants ID and Name.
def getTenants(nb_url, token):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Tenants"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    try:
        # Do the HTTP request
        response = requests.get(full_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        else:
            return ("Get tenants failed! - " + str(response.text))
    except Exception as e: return (str(e))

allT = getTenants(nb_url, token)
"""for i in range(len(allT)):
    print(allT[i].tenantName)"""
allT
 # print out the name of all accessible tenants 

{'tenants': [{'tenantId': 'fb24f3f0-81a7-1929-4b8f-99106c23fa5b',
   'tenantName': 'Initial Tenant'}],
 'statusCode': 790200,
 'statusDescription': 'Success.'}

In [ ]:
# 1. Similar problem with previous API,a little different is in Get all accessible tenants API token is considered as a 
#    query argument (normal parameter).
# 2. Can we combine the accessible domain API with tenant API together. Or add another response of accessible tenants API 
#    which is the number of the accessible domains that the user can access. 

# Get all accessible domains of a tenants API
Use this function returns a list of accessible domains in a specific tenant. The returned accessible domains vary by the user privileges you use to log in. To retrieve a full list of domains in a specified tenant, you must log in with system admin or tenant admin permissions.
<br><br>


* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Domains | Yes |
<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|tenantId | string  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|domains | array | A list of all accessible domains. |
|domainId| string | The domain ID.  |
|domainName| string | The domain name. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |
<br><br>

* **Example**

In [14]:
# Set the request parameters
token = "da156027-0453-4ac4-bcfa-7b0b870b82cc"
nb_url = "http://192.168.28.79"
#tenantId = "fb24f3f0-81a7-1929-4b8f-99106c23fa5b"

# Get all domains info. If tenantId is None, will present all domains.
def getDomains(nb_url, token, tenantId=None):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Domains"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    #headers["Token"]=token
    data = {
                "tenantId": tenantId,
                "token" : token
           }
    try:
        # Do the HTTP request
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        else:
            return ("Get domains failed! - " + str(response.text))
    except Exception as e: return (str(e))
    
allD = getDomains(nb_url, token)
"""for i in range(len(allD)):
    print(allD[i].domainName)"""
allD # print out all accessible tenants in json

{'domains': [], 'statusCode': 790200, 'statusDescription': 'Success.'}

In [ ]:
#---Code from postman
#   GET /ServicesAPI/API/V1/CMDB/Domains?token=da156027-0453-4ac4-bcfa-7b0b870b82cc HTTP/1.1
#   Host: 192.168.28.79
#   cache-control: no-cache
#   Postman-Token: f5358d27-d64a-4bc0-b2ac-46f636c31d43

#---The domains list should provide the domain information which the guest user can access currently. Or users can not keep calling 
#   next API: Specify a domain to work on API. without next step, users cannot doing any further things by calling API.

#---Same Auth token problem.

#---how about combine domain API with tenant API together.

"""Back in 2012, the OAuth specification explicitly said...

"Don't pass bearer tokens in page URLs: Bearer tokens SHOULD NOT be passed in page URLs (for example as query string parameters). 
Instead, bearer tokens SHOULD be passed in HTTP message headers or message bodies for which confidentiality measures are taken."

(https://tools.ietf.org/html/draft-ietf-oauth-v2-bearer-16#section-4.3)

However in the final document for RFC 6750, the "SHOULD NOT" seems to be softened a bit.
The spec seems to recognize the need for exceptions, and there are recommendations for maxiizing security when the 
Authorization header cannot be used."""


# Specify a domain to work on API
Use this API to specify a domain to work on to get or set NetBrain data by associating domainID to the current session.

<br><br>

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **PUT** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session/CurrentDomain | Yes |
<br><br>

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
<br><br>

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|tenantId* | string  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |
|domainId* | string  | input the ID of the target domain. Get a domain ID by using the API [Get all accessible domains of a tenant.](https://www.netbraintech.com/docs/ie71/help/get-all-accessible-domains-of-tenant.htm)   |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
<br><br>

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |
<br><br>

* **Example**

In [16]:
import json
# Set the request parameters
#token = "6c67c683-b891-4584-934d-95e7ccac6c43"
nb_url = "http://192.168.28.79"
#tenantId = "fb24f3f0-81a7-1929-4b8f-99106c23fa5b"
#domainId = "0201adc4-ae96-46f0-ae3d-01cdba9e41d6"#fixed

# Login selected domain
def loginDomain(nb_url, token, tenantId, domainId):
    full_url = nb_url + "/ServicesAPI/API/V1/Session/CurrentDomain"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    body = {
            "tenantId": tenantId,
            "domainId": domainId
        }
    try:
        # Do the HTTP request
        response = requests.put(full_url, data=json.dumps(body), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        elif response.status_code != 200:
            return ("Login failed! - " + str(response.text))
    except Exception as e: return (str(e))

result = loginDomain(nb_url, token, tenantId, domainId)
print(result)# print out the login result of one particular domain. 

{'statusCode': 790200, 'statusDescription': 'Success.'}


In [ ]:
# Code from postmen
"""POST /ServicesAPI/API/V1/Session? HTTP/1.1
Host: 192.168.28.79
Accept: application/json
Content-Type: application/json
cache-control: no-cache
Postman-Token: 5236babb-26bc-496a-91be-4141c9efb470
{
    "username": "gongdailiu",
    "password": "1101810209"
}------WebKitFormBoundary7MA4YWxkTrZu0gW--"""

# Suggestion:
{
    'Message' : "Specific domain connected!"
    'statusCode': 790200, 
    'statusDescription': 'Success.'
}

# Discovery
Uses the following APIs to manage your network discovery tasks in a domain:

>* [Get a list of discovery tasks in a domain](#Get-all-discovery-tasks-in-a-domain)

>* Add IPs as seeds to a discovery task

>* Remove IPs from a discovery task  

>* Get all IPs added to a discovery task

>* Run a discovery task now

>* Get the running status of a discovery task

>* Get the discovery results of devices

>* Get the live access logs of devices

# Get all discovery tasks in a domain
Use this API to get all discovery tasks in the current domain.
>* **Tip:** If there are no discovery tasks in the system, add a new discovery task from the Domain Management page. In the discovery task, select Once on the Frequency tab and Discover the following IPs on the Discovery Seed tab.

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Discovery/Tasks | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|tasks | array | A list of all discovery tasks. |
|id| string | The ID of a discovery task.  |
|name| string | The name of a discovery task. |
|enable| bool | Whether a discovery task is enabled.  |
|lastStatus| string | The last run (result) status of a discovery task. |
|lastRunSpan| integer | The duration of the last run of a discovery task.  |
|curStatus| string | The current status of a discovery task.  |
|nextRunTime| string | The start time of the next run of a discovery task.  |
|frequency| string | The run frequency of a discovery task.  |
|statusCode| integer | The returned status code of executing the API.  |
|statusDescription| string | The explanation of the status code.  |

* **Example**


In [36]:
# Set the request parameters
#token = "647fac47-27eb-4e8f-bde2-ffe647a3f6e6"
#nb_url = "http://192.168.28.79"

# Get list of all discovery tasks and find your task ID
def getDiscoveryTasks(nb_url, token):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Discovery/Tasks"
    try:
        response = requests.get(full_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            return result
        else:
            return ("Get discovery task list failed - " + str(response.text))
    except Exception as e:
        return (str(e)) 

result = getDiscoveryTasks(nb_url, token)
result


{'tasks': [{'id': 'e86e73da-1829-fd8e-4f83-98f524533779',
   'name': 'Scheduled System Discovery',
   'enable': False,
   'lastStatus': '',
   'lastRunSpan': '00:00:00',
   'curStatus': 'Not Started',
   'nextRunTime': '1/2/3000,12:00:00 AM',
   'frequency': 'Once'}],
 'statusCode': 790200,
 'statusDescription': 'Success.'}